In [1]:
# define the model
from keras.models import Model, Sequential
from keras.layers import Input, Dense, concatenate
from keras.layers.normalization import BatchNormalization
from keras.layers.core import Dropout
from keras import optimizers
from keras.utils import to_categorical

img_feature_a = Input(shape=(2048,))
img_feature_b = Input(shape=(2048,))

shared_fc_layer = Sequential([
    Dense(1024, activation='sigmoid', input_shape=(2048, )),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1024, activation='sigmoid'),
    BatchNormalization(),
    Dropout(0.5),
])

encoded_a = shared_fc_layer(img_feature_a)
encoded_b = shared_fc_layer(img_feature_b)

merged_vector = concatenate([encoded_a, encoded_b])

x = Dense(1024, activation='sigmoid')(merged_vector)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output = Dense(2, activation='softmax')(x)

model = Model(inputs=[img_feature_a, img_feature_b], outputs=output)

optimizer = optimizers.RMSprop(lr=1e-3, rho=0.9, epsilon=1e-08, decay=0.0)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1060 6GB (CNMeM is disabled, cuDNN 5110)


In [2]:
import cPickle as pickle
import numpy as np
import os
import random

def load_dataset(pklfile):
    pair_samples = pickle.load(open(pklfile, 'rb'))
    X_1 = np.array(map(lambda x: x[2], pair_samples))
    X_2 = np.array(map(lambda x: x[3], pair_samples))
    Y = np.array(map(lambda x: x[4], pair_samples))
    Y[Y==-1] = 0
    return X_1, X_2, Y

X_1_valid, X_2_valid, Y_valid = load_dataset("./data/201704.pkl")
Y_valid = to_categorical(Y_valid, num_classes=2)

batch_size = 128
pklfile_list = os.listdir("./data/")

for _pass in range(10):
    random.shuffle(pklfile_list)
    for pklfile in pklfile_list:
        #if pklfile < "2017":
        #    continue
        if pklfile == "201704.pkl":
            continue
        print "loading pklfile: %s of pass=%d"%(pklfile, _pass)
        
        X_1, X_2, Y = load_dataset("./data/"+pklfile)
        Y = to_categorical(Y, num_classes=2)

        model.fit([X_1, X_2], Y, batch_size=batch_size, epochs=1, 
                  validation_data=([X_1_valid, X_2_valid], Y_valid), shuffle=True)

loading pklfile: 201702.pkl of pass=0
Train on 62962 samples, validate on 18399 samples
Epoch 1/1
62962/62962 [==============================] - 7s - loss: 0.6306 - acc: 0.7350 - val_loss: 1.3952 - val_acc: 0.5154
loading pklfile: 201701.pkl of pass=0
Train on 48047 samples, validate on 18399 samples
Epoch 1/1
48047/48047 [==============================] - 5s - loss: 0.3478 - acc: 0.8317 - val_loss: 1.9579 - val_acc: 0.5132
loading pklfile: 201703.pkl of pass=0
Train on 69487 samples, validate on 18399 samples
Epoch 1/1
69487/69487 [==============================] - 7s - loss: 0.3355 - acc: 0.8390 - val_loss: 1.7418 - val_acc: 0.5491
loading pklfile: 201701.pkl of pass=1
Train on 48047 samples, validate on 18399 samples
Epoch 1/1
48047/48047 [==============================] - 5s - loss: 0.1440 - acc: 0.9437 - val_loss: 2.1560 - val_acc: 0.5275
loading pklfile: 201703.pkl of pass=1
Train on 69487 samples, validate on 18399 samples
Epoch 1/1
69487/69487 [==============================] -

KeyboardInterrupt: 

In [3]:
import cPickle as pickle
import numpy as np
import os
import random

def load_dataset(pklfile):
    pair_samples = pickle.load(open(pklfile, 'rb'))
    X_1 = np.array(map(lambda x: x[2], pair_samples))
    X_2 = np.array(map(lambda x: x[3], pair_samples))
    Y = np.array(map(lambda x: x[4], pair_samples))
    Y[Y==-1] = 0
    return X_1, X_2, Y

X_1, X_2, Y = load_dataset("./data/sybj.pkl")
Y = to_categorical(Y, num_classes=2)
X_1_valid, X_2_valid, Y_valid = load_dataset("./data/sybj.valid.pkl")
Y_valid = to_categorical(Y_valid, num_classes=2)

batch_size = 128

model.fit([X_1, X_2], Y, batch_size=batch_size, epochs=10, 
            validation_data=([X_1_valid, X_2_valid], Y_valid), shuffle=True)

Train on 930400 samples, validate on 20418 samples
Epoch 1/10
930400/930400 [==============================] - 393s - loss: 0.5464 - acc: 0.7202 - val_loss: 0.8311 - val_acc: 0.6029
Epoch 2/10
930400/930400 [==============================] - 163s - loss: 0.3172 - acc: 0.8610 - val_loss: 0.9827 - val_acc: 0.6085
Epoch 3/10
930400/930400 [==============================] - 102s - loss: 0.2426 - acc: 0.8976 - val_loss: 1.1392 - val_acc: 0.5994
Epoch 4/10
930400/930400 [==============================] - 99s - loss: 0.2089 - acc: 0.9134 - val_loss: 1.1105 - val_acc: 0.6203
Epoch 5/10
930400/930400 [==============================] - 99s - loss: 0.1873 - acc: 0.9232 - val_loss: 1.1599 - val_acc: 0.6239
Epoch 6/10
930400/930400 [==============================] - 99s - loss: 0.1727 - acc: 0.9295 - val_loss: 1.2715 - val_acc: 0.6000
Epoch 7/10
930400/930400 [==============================] - 99s - loss: 0.1636 - acc: 0.9336 - val_loss: 1.2072 - val_acc: 0.6242
Epoch 8/10
930400/930400 [==========

KeyboardInterrupt: 

In [ ]:
print X_1_valid.shape
print X_2_valid.shape
print Y_valid.shape

In [4]:
X_1_valid, X_2_valid, Y_valid = load_dataset("./data/201704.pkl")

In [5]:
Y_predict = model.predict([X_1_valid, X_2_valid])

In [8]:
%matplotlib inline
from PIL import Image
import matplotlib.pyplot as plt

#plt.plot(Y_valid[:,1], Y_predict[:,1], 'bo')
np.corrcoef([Y_valid[:], Y_predict[:,1]])

array([[ 1.        ,  0.17752544],
       [ 0.17752544,  1.        ]])